# Logging进阶篇  

和基础篇比，本篇才算是打开了logging的大门，基础篇的很多操作其实内部都做了不少默认动作，尤其是basicConfig，所做的内容其实很多  
本篇从基础介绍了logging 模块的基本类，通过对基础类的实例化来创建日志记录器，虽然相对基础篇更复杂，但其实更加符合python的思维。

logging 底层有四个大类：
* Logger: 也就是日志记录器类，实例化后直接用来操作各种日志记录操作
* Handler: 用于确定将日志记录输出到哪里，一个日志记录器可以有多个handler
* Filter: 更细粒度的区分对什么日志进行输出
* Formatter: 更细粒度的确定日志输出格式

最终日志记录器生成一个 LogRecord 实例，从而完成日志输出

## 整个logging的流程（非常重要）

![LoggingFlow](https://docs.python.org/3/_images/logging_flow.png)

流程图可以表述成以下内容：
* logging call (请求日志记录)
* 创建 LogRecord 实例
* 判断日志记录器的 filter 是否拒绝了该记录？
* 将日志记录传输给该日志记录器的所有handlers
    * handlers 判断 LogRecord 是否符合自己的level
    * handlers 的 filters 是否命中了该条 LogRecord
    * 输出
* 判断该日志记录器，是否有传输（广播）的性质
* 如果有，判断logger 是否有父logger，有的话将LogRecord继续传给父logger的handlers

所谓父logger，在Logger会讨论

## Logger  

* 实例化对象，就是logger，日志记录器
* 每个日志记录器都需要有个名字
    * 命名规则是可以有继承关系的，比如名字为 scan 的日志记录器，就是名字为 scan.pdf, scan.text 的父logger，当调用scan.pdf记录日志的时候，LogRecord同样会被传入给scan的handlers，被判断要不要被输出在一些地方（这个功能太高级了，暂时用不到）
    * 继承规则有一个用处是确定该logger的默认等级，当这个logger的等级没有被明确是什么的时候，会尝试去寻找父节点的level是什么，一直追查到最终根节点的logger，根节点肯定会有明确的level
* 名字原则上没有任何要求，仅仅是用来代表你知道这个日志出自哪里
* 但是最常见的命名logger的方式还是： logger = logging.getLogger(\_\_name\_\_) 因为这样可以明确的知道这条日志来自哪里
* 既然日志记录器的命名是可以继承的，那么就存在一个根节点的名字，根节点的名字就叫做 'root'，如果直接用logging来call，默认就是用这个logger
* logger 用logging.getLogger(name) 的形式获得，如果这个 name 在整个进程（全局）中已经被创建过，则会返回原有的logger，否则会创建一个新的logger，但是 'root' 这个字符除外，他不会调用根节点的logger， 仍然会创建一个新的，名字也叫 root 的子节点logger，因此实际中不建议创建名字为 root 的日志记录器


In [1]:
import logging
# 直接logging的日志记录器，默认的形式
logging.warning('This is a default warning')

In [1]:
# 需要重启解释器
import logging 
# 名字为__name__的日志记录器，默认的形式'
logger = logging.getLogger(__name__)
logger.warning('This is a __name__ warning')

# 名字为 root 的日志记录器，格式与上面的一致
logger_r = logging.getLogger('root')
logger_r.warning('This is a warning by a logger named root')

# logging 默认的根节点日志
logging.warning('This is a default warning')

This is a __name__ warning
This is a warning by a logger named root


可以看到 logger_r 和 根节点日志记录器，并不是同一个日志记录器

### Logger的工作内容，有个三个层面
* 日志各种 call
* 确定一个基本的输出 level：类似 warning，info 等 （其实就是一个基本的filter）
* 将满足level的log传输给相应的handler

这里可以看到，除了不能绑定 format，logger 可以与 filter和hanler 绑定完成工作

### 最常用的logger的函数
* Logger.setLevel(): 确定输出level
* Logger.addHandler() / Logger.removeHandler(): 添加 Handler
* Logger.addFilter() / Logger.removeFilter(): 添加Filter

当然你不需要每次创建新的日志记录器的时候都调用这些方法：
* 在python 3.2 之后，如果没有指定过handler，那么会使用 logging.lastResort，但是这不代表 logging.lastResort 被绑定在该logger上了
* 如果没有 setLevel 或者 指定过 filter ，那么logger本身的筛选就是none，对所有的信息都可以通过，真正起作用的是各个 handler 中的 filter，也就是level

In [2]:
logging.lastResort

<_StderrHandler stderr (WARNING)>

### 当Logger被配置完成以后
* Logger.debug()/Logger.info()/Logger.warning()/Logger.error()/Logger.critical()
    * 创建相应的 LogRecord
    * 有一个参数： exc_info 是用来判定log是否要输出 exception 信息的
* Logger.expection()
    * 和 logger.error() 相似，区别就在于exception 同时会输出 stack trace
    * 只有 exception handler 才可以用
* Logger.log()
    * 可以设定 loglevel，这个loglevel应该是可以用数字的，适用于整个日志等级不仅仅是简单的五个level，需要更多等级时候的个性化配置

In [1]:
import logging
# 验证 logging.warning logging.error logging.exception 的关系
try:
    int('s')
except:
    logging.exception('error:')

try:
    int('s')
except:
    logging.error('error but no exception')

try:
    int('s')
except:
    logging.error('error with exception:', exc_info=True)

try:
    int('s')
except:
    logging.error('error but no exception', exc_info=False)

try:
    int('s')
except:
    logging.warning('warning with exception', exc_info=True)

ERROR:root:error:
Traceback (most recent call last):
  File "<ipython-input-1-5ce02062a8d4>", line 4, in <module>
    int('s')
ValueError: invalid literal for int() with base 10: 's'
ERROR:root:error but no exception
ERROR:root:error with exception:
Traceback (most recent call last):
  File "<ipython-input-1-5ce02062a8d4>", line 14, in <module>
    int('s')
ValueError: invalid literal for int() with base 10: 's'
ERROR:root:error but no exception
Traceback (most recent call last):
  File "<ipython-input-1-5ce02062a8d4>", line 24, in <module>
    int('s')
ValueError: invalid literal for int() with base 10: 's'


### 关于传播
* 子日志记录器会把LogRecord传递给父日志记录器，所以没有必要将所有的handlers赋给所有的logger，合理利用这种层级关系，可以更有效
* 如果要修改这种传播性质，可以用 Logger.propagate
* 这里要注意的是，logger如果没有addHandler过，那么会调用logging.lastResort 来做输出，这并不代表handler被绑定在logger上了

In [1]:
# 需要重启解释器
# 新日志记录器的父记录器是 root，并且有传播的性质
import logging

# 创建一个日志记录器
logger = logging.getLogger('foo')
# 会打印一条信息，虽然有传播性质，但是因为 root 没有初始化，也就是没有绑定过handler，所以不会输出
logger.warning('warning from foo')

warning from foo


In [2]:
# 给根记录器也绑定一个handler
logging.basicConfig(format = '%(name)s: %(message)s',level = 'INFO')
logging.info('init root')

root: init root


In [3]:
# 会发送一条，且仅来自root，因为 logging.lastResort 并没有被使用
logger.warning('warning from foo')

foo: warning from foo


In [4]:
# 给 logger 也绑定handler，方便起见就使用 logging.lastResort
logger.addHandler(logging.lastResort)
# 这里会发送两条，一条来自 foo， 一条来自 root
logger.warning('warning from foo')
# 另外我们可以看到因为本身这个 LogRecord 是有 foo 创建的，所以虽然是root发出的信息，还是会显示 name = foo

warning from foo
foo: warning from foo


In [5]:
# 这里会出现一条，是来自 root 的，因为 foo 自己并没有设置 filter，info信息是可以通过的，之所以没输出是 foo对应的handler限制了输出
logger.info('info from foo')

foo: info from foo


In [6]:
# 对 foo setLevel
logger.setLevel('WARNING')
# 这里就不会出现任何记录，因为logger在最开始就过滤掉了本条记录，没有传给root
logger.info('info from foo')

## Handler

* 一个日志记录器可以把 LogRecord 传输给不同的输出内容，包括
    * 文件
    * HTTP GET/POST 
    * 邮件
    * sockets
    * queues (可能是进程之间的管道？)
    * 系统特定日志记录方法
    * ……任何其他自定义的handler
* 默认状态下没有handle会直接被赋予一个logger，如果使用basicConfig，那么会有一个传输给 console 或者 文件 的handle赋予root
* 如果没有使用basicConfig 直接使用了各种call 命令，则会创建一个到 console 的handler(当然也会创建一个默认的format)
* 正如流程图所示，每个handler自己是可以有filter 的，也就是说你可以给一个 LogRecord 设置多个 handler，每个handler有不同的筛选条件，比如将所有的日志信息输入文件，但是仅在 console 输出warning 以上的信息

### Handle 标准操作

其他的操作，因不同的Handler 会有不同
* setLevel(): 设置level
* setFormatter(): 设置 format
* addFilter() / removeFilter(): 设置特殊的Filter

### 标准库 Handler 列举

各个Handler的使用方法参考各种接口

* StreamHandler: 传递数据给console，默认是 sys.stderr， 可以指定其他的流，可以直接在basicConfig中指定
* FileHandler: 可以设定文件，写入的方式（a，w），编码，延迟（直到有第一个call才创建文件），可以直接在basicConfig中指定
* BaseRotatingHandler: 循环文件的Base类，不太需要直接使用
    * RotatingFileHandler: 可以循环写入文件，按照文件的大小分割（默认文件大小可以无限大），每次被写的文件都是 .log ，当rollover时，会被命名成 .log.1 其他的名字会被重命名到 .2  .3 ...
    * TimedRotatingFileHandler: 可以循环写入文件，按照时间分割，可以按照 各种 时间间隔 来间隔，也可以用 'midnight' , 可以设置文件后缀,具体如果需要了解还是官方文档的好。
* SocketHandler
* ……

## Formatter

Formatter 就是用来控制日志最终顺序，结构和内容的类，如果没有指定过Formatter，会自动生成（并挂载？）

### 参数
* message format string: 整个信息的格式，默认就是没有纯粹的信息文本，不带任何其他东西
* date format string: 日期的表现格式， %Y-%m-%d %H:%M:%S
* style indicator: 用来确定 message format string 中的变量是用什么格式表现的，可选的比如 "%" "{" "%"
    * 注意这里的各种style并不表示传入自定义变量时可以用的格式，自定义变量的传入永远 用 %-style 传入，除非自己先合并再整体传入

In [1]:
import logging
from logging import Formatter,StreamHandler


# formatter，用两种style
formatter_1 = Formatter(fmt='%(asctime)s - %(levelname)s - %(name)s - %(message)s', datefmt='%H:%M:%S', style = '%')
# 之所以要提供 {} 格式，是因为 .format 可以控制输出的更多选项，比如 {msecs:03d} 
formatter_2 = Formatter(fmt='{asctime} - {levelname} - {name} - {message}', datefmt='%H:%M:%S', style = '{')

# 创建 handler
handler_1 = StreamHandler()
handler_1.setFormatter(formatter_1)
handler_2 = StreamHandler()
handler_2.setFormatter(formatter_2)

# 创建日志记录器
logger_1 = logging.getLogger('foo')
logger_1.addHandler(handler_1)
logger_2 = logging.getLogger('bar')
logger_2.addHandler(handler_2)

In [2]:
# logger_1
logger_1.warning('This is a %s from foo','warning')

11:33:56 - WARNING - foo - This is a warning from foo


In [3]:
# 并不支持 format 格式的自定义变量传入，但是可以提前自己合并(直接写，或者写函数将 (*,*,*)的参数先合并到 str.format 中  )
logger_2.warning('This is a {0} from bar'.format('warning'))

11:33:57 - WARNING - bar - This is a warning from bar


### 时区转换

时区默认用的是本地时区，如果需要转换成标准时区，需要改变 formatter 中 converter 的值

In [1]:
# 重启解释器

import time
import logging
from logging import Formatter,StreamHandler

formatter_1 = Formatter(fmt='%(asctime)s - %(levelname)s - %(name)s - %(message)s', datefmt='%H:%M:%S')
formatter_2 = Formatter(fmt='%(asctime)s - %(levelname)s - %(name)s - %(message)s', datefmt='%H:%M:%S')
# 修改converter
formatter_2.converter = time.gmtime

handler_1 = StreamHandler()
handler_1.setFormatter(formatter_1)
handler_2 = StreamHandler()
handler_2.setFormatter(formatter_2)

logger_1 = logging.getLogger('foo')
logger_1.addHandler(handler_1)
logger_2 = logging.getLogger('bar')
logger_2.addHandler(handler_2)

In [2]:
logger_1.warning('This is a warning from foo with localtime')
logger_2.warning('This is a warning from bar with GMT time')

14:46:32 - WARNING - foo - This is a warning from foo with localtime
06:46:32 - WARNING - bar - This is a warning from bar with GMT time


## Filter

教程里面没有Filter 的直接介绍，不过为了保证介绍的完整性，特地了解了一下

* 之所以没有是因为，Filter的概念已经变的比较模糊了，他就是一个过滤器，可以是一个带 filter 函数功能的class，也可以就是一个函数，函数执行了过滤器的功能
* 不论是Filter.filter() 还是自定义的一个函数，接收的都是一个 [LogRecord](https://docs.python.org/3/library/logging.html#logging.LogRecord) 的实例
* Filter(name='')，这里的名字指的是logger的名字，并且包含他的子logger

In [3]:
def filter_engine(record):
    '''过滤含有 hello 的log'''
    if 'hello' in record.msg:
        return False
    return True

logger_1.addFilter(filter_engine)
logger_1.warning('Hello from foo')     
# 没有任何信息输出
logger_1.warning('hello from foo')

14:46:35 - WARNING - foo - Hello from foo


## 日志属性配置

我们有三种方法配置日志的属性
* 代码创建 logger, handler, formatter并且自己绑定好
* 创建一个 file，里面包含了所有的配置，并且用 fileConfig() 读取
* 创建一个字典型配置变量，用 dictConfig() 读取

In [1]:
# 需要重启解释器
# 第一种方案
import logging

# create logger
logger = logging.getLogger('simple_example')
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

2019-01-31 15:14:32,643 - simple_example - DEBUG - debug message
2019-01-31 15:14:32,645 - simple_example - INFO - info message
2019-01-31 15:14:32,646 - simple_example - WARNING - warn message
2019-01-31 15:14:32,648 - simple_example - ERROR - error message
2019-01-31 15:14:32,650 - simple_example - CRITICAL - critical message


### logging.conf

[loggers]  
keys=root,simpleExample  

[handlers]  
keys=consoleHandler  

[formatters]  
keys=simpleFormatter  

[logger_root]  
level=DEBUG  
handlers=consoleHandler  

[logger_simpleExample]  
level=DEBUG  
handlers=consoleHandler  
qualname=simpleExample  
propagate=0  

[handler_consoleHandler]  
class=StreamHandler  
level=DEBUG  
formatter=simpleFormatter  
args=(sys.stdout,)  

[formatter_simpleFormatter]  
format=%(asctime)s - %(name)s - %(levelname)s - %(message)s  
datefmt=  

In [1]:
# 需要重启解释器
import logging
import logging.config

logging.config.fileConfig('logging.conf')

# create logger
logger = logging.getLogger('simpleExample')

# 'application' code
logger.debug('debug message')
logger.info('info message')
logger.warning('warn message')
logger.error('error message')
logger.critical('critical message')

2019-01-31 15:27:44,320 - simpleExample - DEBUG - debug message
2019-01-31 15:27:44,323 - simpleExample - INFO - info message
2019-01-31 15:27:44,326 - simpleExample - WARNING - warn message
2019-01-31 15:27:44,329 - simpleExample - ERROR - error message
2019-01-31 15:27:44,331 - simpleExample - CRITICAL - critical message


用config的优势就是明晰，要改动也相对方便  
_使用 fileConfig 和 dictConfig 都会默认将 disable existing loggers 设置为True，也就是在file之前的logger全部停用，可以根据自己的需要查看官方文档如何恢复_

### 使用字典（最推荐的方法）
Python 3.2 之后，可以使用字典的方法配置logger，相对更方便可读  

以及官方推荐的字典的书写方式是用 yaml，这个语言相对json貌似更可读   

具体字典的写法，参照：[logging-config-dictschema](https://docs.python.org/3/library/logging.config.html#logging-config-dictschema)

In [1]:
import yaml

with open('logging.yaml') as f:
    conf = yaml.load(f)
print(conf)

{'version': 1, 'formatters': {'simple': {'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s'}}, 'handlers': {'console': {'class': 'logging.StreamHandler', 'level': 'DEBUG', 'formatter': 'simple', 'stream': 'ext://sys.stdout'}}, 'loggers': {'simpleExample': {'level': 'DEBUG', 'handlers': ['console'], 'propagate': False}}, 'root': {'level': 'DEBUG', 'handlers': ['console']}}


In [2]:
import logging
import logging.config

logging.config.dictConfig(conf)

logger = logging.getLogger('simpleExample')

logger.info('Info Test')

2019-01-31 16:00:52,599 - simpleExample - INFO - Info Test


## 实例编写一个 root 的yaml配置文件

经过整个学习以后，发现可以充分的利用日志记录器的层级关系和传播性质，将除 streamhandler 以外的所有 handler 都创建在 logging.root 里，子日志记录器其实只要出个名字就可以了，这样的日志管理系统应该更简洁，而且对除了 stream 以外的输出都由 logging.root 负责

In [1]:
# 重启解释器
import yaml
import logging
import logging.config

with open('log_root.yaml') as f:
    conf = yaml.load(f)
logging.config.dictConfig(conf)

In [2]:
logging.debug('这条记录只会打印在console,不会记录在日志')

2019-01-31 16:47:11 - root - DEBUG - 这条记录只会打印在console,不会记录在日志


In [3]:
logging.info('这条记录既会打印在console, 又会打印在日志')

2019-01-31 16:47:12 - root - INFO - 这条记录既会打印在console, 又会打印在日志


In [4]:
# 获得一个logger，不绑定任何handlers，filters，此时只会运行root绑定的handlers，但是name显示的是__name__
logger = logging.getLogger(__name__)
logger.debug('这条记录只会打印在console,不会记录在日志')
logger.info('这条记录既会打印在console, 又会打印在日志')

2019-01-31 16:47:13 - __main__ - DEBUG - 这条记录只会打印在console,不会记录在日志
2019-01-31 16:47:13 - __main__ - INFO - 这条记录既会打印在console, 又会打印在日志


In [5]:
# 还可以在logger这层实现过滤，filters 或者是 setLevel
logger = logging.getLogger(__name__)
logger.setLevel('WARN')
logger.info('这条记录不会输出任何信息，也不会被root捕捉到')
logger.warning('这条记录会被输出，同时也会被捕捉到，且使用的还是 root 的handlers')

2019-01-31 16:47:15 - __main__ - WARNING - 这条记录会被输出，同时也会被捕捉到，且使用的还是 root 的handlers
